In [ ]:
!git clone https://github.com/AgPriyank/react_agent

In [ ]:
!pip install openai

In [ ]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
from openai import OpenAI
client = OpenAI()

In [ ]:
import os
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
responses=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role":"user","content":"Say Hello"}]
)

print(responses.choices[0].message.content)


In [ ]:
client =OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
responses=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role":"user","content":"What is 2+2?"}]
)
print(responses.choices[0].message.content)

In [ ]:
def askllm(question):
  responses=client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{"role":"user","content":question}]
  )
  return responses.choices[0].message.content

In [ ]:
askllm("What is the capital of India?")

In [ ]:
### text parsing text contains Action and Action input

def parse_text(text):
  text=text.lower()
  lines=text.split("\n")
  for line in lines:
    line_strip=line.split(": ")
    if len(line_strip)!=2:
      continue
    if "action"==line_strip[0]:
      action=line_strip[1]
    elif "action input" ==line_strip[0]:
      action_input=line_strip[1]
    elif "thought" ==line_strip[0]:
      thought=line_strip[1]
  return action,action_input, thought

In [ ]:
text="""Thought: I need to search for information.
Action: search
Action Input: Paris population"""
parse_text(text)

In [ ]:
text2 = """
Thought: I should search for the population
Action: search
Action Input: Paris population
"""

text1 = """
Thought: I need to calculate
Action: calculator
Action Input: 2 + 2
"""

text4 = """
Thought: I need to compute the percentage
Action: calculator
Action Input: 150 * 0.20 """

In [ ]:
parse_text(text4)

In [ ]:
!pip install asteval

In [ ]:
from asteval import Interpreter
aeval = Interpreter()
import re
def calculator(expression):
    """
    Takes a math expression as string, returns result.
    Example: calculator("2 + 2") -> 4
    """
    expression=re.sub(r"\s*of\s*","*",expression)
    expression= re.sub(r'(\d+)%', r'\1/100', expression)
    try:
      answer = aeval(expression)
      return str(answer)
    except:
      return "Invalid expression"


In [ ]:
calculator("10% of 200")

In [ ]:
def search(query):
    """
    Pretend to search, return fake results.
    """
    fake_db = {
        "paris": "Paris is the capital of France with a population of 2.2 million",
        "france": "France is a country in Europe. Its capital is Paris",
        "capital": "The capital of France is Paris"
    }

    query_lower= query.lower()

    for key in fake_db.keys():
      if key in query_lower:
        return fake_db[key]
    else:
      return "not found"



In [ ]:
question="What is the population of the capital of France?"
prompt=f"""Answer in the following format

Thought: think about what to do
Action: one of [search, calculator, finish]
Action input: input for the action


IMPORTANT: Output only ONE action at a time, then STOP. Wait for the Observation before continuing.

Available actions:
- search: look up information
- calculator: evaluate a math expression
- finish: give the final answer

Question: {question}"""

import os
from openai import OpenAI

client=OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))


n=3
for i in range(n):
  print(f"interation\n---{i+1}----\n")

  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{"role":"user","content":prompt}],
      stop=["Observation"],
      temperature=0
  )
  output=response.choices[0].message.content
  print(f"complete output\n--{output}\n")

  action,action_input, thought = parse_text(output)

  if action == "finish":
    print(f"final answer : {action_input}\n")
    break
  elif action == "calculator":
    observation=calculator(action_input)
  elif action =="search":
    observation=search(action_input)
  else:
    print(f"invalid action {action}\n")

  print(f"observation {observation}\n")

  prompt = prompt + f"""\n{output}\n Observation: {observation}\n If you think you are done then reply with
  Thought: I am done
  Action: Finish
  Action Input: Give the last observation as the final answer"""


